<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, clear_output
import time
def sleepyTime():
    count=0
    try:
        while count <= 120:
            print(f'sleeping for {count} minutes')
            time.sleep(60)
            count+=1
            if count % 5 == 0:
                clear_output()
    except KeyboardInterrupt:
        clear_output()
# sleepyTime()

In [ ]:
from __future__ import absolute_import
from IPython.display import display, clear_output

try: import tensorflow as tf
except ModuleNotFoundError:
    %pip install tensorflow
    import tensorflow as tf
    
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import smart_resize, save_img 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

try: import matplotlib.pyplot as plt
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib.pyplot as plt

try: import pandas as pd
except ModuleNotFoundError:
    %pip install pandas
    import pandas as pd

try: 
    import numpy as np
    np.set_printoptions(precision=4)
except ModuleNotFoundError:
    %pip install numpy
    import numpy as np
    np.set_printoptions(precision=4)

try: import PIL.Image
except Exception as err:
    print(err)
    %pip install pillow
    import PIL.Image

try: import scipy.ndimage as ndimage
except Exception as err:
    print(err)
    %pip install SciPy
    import scipy.ndimage as ndimage

import glob
import pathlib
import os
from os.path import *
import shutil

contentPth = os.getcwd()

if contentPth == '/content':
    gdrivePth = join(contentPth, 'gdrive')
    myDrivePth = join(gdrivePth, 'My Drive')
    tfImagesPth = join(contentPth, 'TensorflowImages')
    gdriveImagesPth = join(myDrivePth, 'TensorflowImages')
    # print(f'tfImagesPth: {tfImagesPth}')
    # print(f'gdriveImagesPth: {gdriveImagesPth}')

    if not exists(tfImagesPth) and exists(contentImagesPth):
        shutil.copytree(src=gdriveImagesPth, dst=tfImagesPth)
else: pass

tfImagesPth = join(contentPth, 'TensorflowImages')

if os.path.exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

try:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/gdrive', force_remount=True)
    os.chdir(myDrivePth)
    from BashColors import C
    from TarfileFunctions import tff
    os.chdir(contentPth)
except ModuleNotFoundError:
    os.chdir(contentPth)
    from BashColors import C
    from TarfileFunctions import tff
    tfImagesPth = join(contentPth, 'TensorflowImages')

if not exists(tfImagesPth):
    try:
        tff.listTarfiles()
        tff.extractTarfiles('Defcon_Images.tar.gz')
    except FileNotFoundError as err:
        # print(f'{err}\n')
        pass
else:
    print(f'exists: {C.Green}{tfImagesPth}{C.ColorOff}')

os.chdir(contentPth)
print(f'cwd: {C.IBlue}{pathlib.Path.cwd()}')

In [ ]:
os.getcwd()

In [ ]:
def tensorflowRot90():
    os.chdir(tfImagesPth)
    imageList = glob.glob('**', recursive=True)
    for thisImage in sorted(imageList):
        fullPath = abspath(thisImage)
        if isfile(thisImage) and thisImage.endswith('.png'):
            # image = tf.keras.preprocessing.image.smart_resize(x=thisImage, size=(224, 224))
            image = tf.keras.preprocessing.image.load_img(
                path=thisImage, grayscale=False, color_mode="rgb",
                target_size=(224, 224), interpolation="nearest"
            )

            display(image)
            print(basename(thisImage))
            print(image.__class__)

            newImage = tf.keras.preprocessing.image.load_img(
                path=thisImage, grayscale=False, color_mode="rgb",
                target_size=(224, 224), interpolation="nearest"
            )

            newImage = img_to_array(newImage)
            rotatedImage = tf.image.rot90(newImage)
            rotatedImage = array_to_img(x=rotatedImage)
            display(rotatedImage)

            rotatedImageName = 'rot90_' + str(basename(thisImage))
            print(rotatedImageName)

            split = os.path.split(p=fullPath)
            directoryPth = split[0]
            savePth = join(directoryPth, rotatedImageName)
            if not exists(savePth):
                pass
                # rotatedImage.save(fp=savePth)
            else: print(f'exists: {C.Green}{savePth}{C.ColorOff}')

tensorflowRot90()

In [ ]:
from PIL import Image
from tensorflow.keras.preprocessing.image import smart_resize, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

os.chdir(tfImagesPth)
globFileList = glob.glob('**', recursive=True)

for fil in sorted(globFileList):
    if isfile(fil):
        fullPath = abspath(fil)
        imageName = basename(fullPath)

        loaded_image = load_img(path=fullPath)
        loaded_image = img_to_array(loaded_image)

        resizedImage = smart_resize(x=loaded_image, size=(224, 224))
        resizedImage

        display(array_to_img(x=resizedImage, scale=True))
        print(imageName)
        print(resizedImage.__class__)



In [ ]:

defconDataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=tfImagesPth,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(224, 224),
    shuffle=True,
    seed=456,
    validation_split=None,
    subset=None,
    # interpolation="bilinear",
    interpolation='mitchellcubic',
    follow_links=False,
    # smart_resize=True,
)
defconDataset.element_spec

In [ ]:
os.chdir(contentPth)
modelList = glob.glob('*.h5')
modelList

In [ ]:
try:
    modelPth = join(
        contentPth, 'mobilenetv2_1.00_None_no_weights_no_top.h5')
    
    mobilenetv2 = tf.keras.models.load_model(
        filepath=modelPth,
        custom_objects=None, compile=True, options=None)

except OSError as err:
    # print(err)
    modelPth = join(
        myDrivePth,'mobilenetv2_1.00_no_weights_no_top.h5')
    mobilenetv2 = tf.keras.models.load_model(
        filepath=modelPth,
        custom_objects=None, compile=True, options=None)
    
# mobilenetv2.summary()

In [ ]:
def modelInfo(model):
    print(f'model name: {C.ICyan}{model.name}{C.ColorOff}')
    print(f'input_shape: {C.ICyan}{model.input_shape}{C.ColorOff}')
    print(f'output_shape: {C.ICyan}{model.compute_output_shape(input_shape=(None, 224, 224, 3))}{C.ColorOff}')
    
modelInfo(model=mobilenetv2)

In [ ]:
augmentedDirectoryPth = join(contentPth, 'AugmentedImages')
if not exists(augmentedDirectoryPth):
    os.makedirs(augmentedDirectoryPth)
    
    ImageDataGenerator.flow_from_directory(
    directory=tfImagesPth,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=1,
    shuffle=True,
    seed=456,
    save_to_dir=augmentedDirectoryPth,
    save_prefix="augmented",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

In [ ]:
image_gen = ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=179,
    width_shift_range=0.0, height_shift_range=0.0, 
    brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=True, vertical_flip=True, rescale=1.0/255,
    preprocessing_function=None, data_format=None,
    validation_split=0.0, dtype=None
)